### Figure 2C - Barnyard Percentage Genome Plots

Here we are going to look at what percentage of each of the barnyard experiments uniquely maps to either the B.sub genome or to the E. coli genome.  

In [1]:
import pandas as pd
import os
from bokeh.io import push_notebook, show, output_notebook, export_svg
output_notebook()
from bokeh.plotting import figure
import datetime

Loading BokehJS ...

First we will read our data frames into a file:

In [2]:
df_dir = '../../../Processed Data Files/dataframe files/230117Li/'
prefix = '230117Li_Mixing_'
df_dict = dict()
names = []
# Fetch all the relevant dataframe files and put them in a useable format:
for file in os.listdir(df_dir):
    if file.endswith('_dataframe.txt') and file.startswith(prefix):
        name = file[file.find(prefix)+len(prefix):]
        name = name[:name.find('_1')]
        new_df  = pd.read_csv(''.join([df_dir,file]))
        new_df['Length'] = new_df['Stop'] - new_df['Start']
        df_dict[name] = new_df
        names.append(name)
print(f'\nHere is the list of unique samples for which there are dataframe files: \n\n{names}\n')

# a name dictionary for converting between the datafram name and a form nicer for plots:
names_dict = {'Ecoli_Cap':'E. coli\nCapsule', 'Mix_Cap':'Mixture\nCapsule', 'Ecoli_Lysate':'E. coli\nLysate', 
              'Mix_Lysate':'Mixture\nLysate', 'Bsub_Lysate':'B. subtilis\nLysate'}


Here is the list of unique samples for which there are dataframe files: 

['Bsub_Lysate', 'Ecoli_Cap', 'Ecoli_Lysate', 'Mix_Cap', 'Mix_Lysate']



Some helper functions to help us with the plotting:

In [3]:
def into_RPM(df_dict, name):
    '''This felper function gets the dataframe for a particular experiment and converts its reads into RPM'''
    sample_df = df_dict[name].copy()
    sample_df = sample_df.rename(columns = {'Counts':name})
    sample_df[name] = sample_df[name]/(sum(sample_df[name])/1000000) #RPM]
    
    return sample_df

Lets get the data we need in order to look at the comparative abundance between the two capsule samples.  This notebook will output both the an interactive plot below and an svg image of the plot.  In order for the svg export to work you need to have selenium and the firefox/geckodriver libraries installed via conda.  Feel free to comment these parts of the code out if all you want is the interactive plot. 

In [4]:
#We will map all reads mapping to the capsule reads as being part of the E.coli genome as well:
genome_map = {'bsubtilis':'B. subtilis','capsule':'E. coli', 'ecoli':'E. coli'}
real_names = []
data = {'samples': real_names}
genomes = list(set(genome_map.values()))
plot_names = ['Mix_Lysate', 'Mix_Cap']
    
for n in plot_names:
    df = df_dict[n].copy()
    df["Genome"] = df["Genome"].map(genome_map)
    real_names.append(names_dict[n])
    for g in genomes:
        arr = []
        if g in data.keys():
            arr = data[g]
        arr.append(100*(sum(df[df.Genome == g].Counts)/sum(df.Counts)))
        data[g] = arr
percent_b_sub = [f'{int(x*1000)/1000.}%\nB. sub' for x in data['B. subtilis']]

Plot the data - for each point there is a bar chart illustrating what percentage of reads uniquely maps to either the E.coli or the B. subtilis genome:

In [6]:
p = figure(y_range = [0, 110], x_range = real_names, height=700, width = 1700, output_backend = "svg", title="Percent Reads Mapping to Each Genome")
colors = ['lightseagreen', 'lightsalmon', 'lightpink', 'khaki']
p.vbar_stack(genomes, x = 'samples', width=0.9, source=data, color = colors[:len(genomes)],
             legend_label=genomes)
p.text(x = real_names, y = [10 for _ in real_names], text = percent_b_sub, align = "center", font_size = '35pt', text_font = ["arial" for _ in real_names])

p.title.text_font_size = "50pt"
p.y_range.start = 0
p.x_range.range_padding = 0.1
p.xgrid.grid_line_color = None
p.axis.minor_tick_line_color = None
p.outline_line_color = None
p.legend.location = (200, 615)
p.legend.label_text_font_size = '35pt'
p.legend.orientation = "horizontal"

p.title.text_font_size = "50pt"
p.title.align = 'center'

p.xaxis.major_label_text_font_size = "45pt"
p.xaxis.major_label_text_font = "arial"

p.yaxis.axis_label="Percentage"

p.yaxis.axis_label_text_font_size = "45pt"
p.yaxis.axis_label_text_font_style = "normal"
p.yaxis.major_label_text_font_size = "35pt"

show(p)
export_svg(p, filename = f'./2C_{datetime.date.today()}.svg')

['./2C_2023-07-03.svg']